In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class QuadTree():
    def __init__(self):
        self.nw = None
        self.sw = None
        self.ne = None
        self.se = None

In [3]:
root = QuadTree()

In [5]:
root.nw = QuadTree()

In [7]:
root.nw